In [ ]:
#!/usr/bin/python

import sys
import os.path
import time
import threading
import requests
from pymodbus.client.sync import ModbusTcpClient
from influxdb import InfluxDBClient

def print_help():
    print "This is a script to gather Modbus points and store them in an InfluxDB database"
    print "It accepts a file that defines the database and what points to poll"

def parse_configuration(conf_file):
    with open(conf_file) as config:
        influx = config.readline().strip().split('|')
        modbus = []
        for line in config:
            modbus.append(line.strip().split('|'))
        return influx, modbus

def data_worker(influx_client, index, worker_count, modbus_conf):
    chunksize = len(modbus_conf)/worker_count
    if index < len(modbus_conf)%worker_count:
        chunksize += 1
        index = index*chunksize
    else:
        index = index*chunksize+len(modbus_conf)%worker_count
    for i in range(index, index+chunksize):
        modbus_client = ModbusTcpClient(modbus_conf[i][0])
        modbus_data = modbus_client.read_holding_registers(int(modbus_conf[i][2]), 1,
                                                           unit=int(modbus_conf[i][1]))
        modbus_data = modbus_data.getRegister(0)
        modbus_client.close()
        data_point = [
            {
                "measurement": "modbus_test",
                "tags": {
                    "IP": modbus_conf[i][0],
                    "Modbus id": modbus_conf[i][1],
                    "Name": modbus_conf[i][3],
                    "Modbus register": modbus_conf[i][2]
                },
                "fields": {
                    "value": modbus_data
                }
            }
        ]
        influx_client.write_points(data_point)

def main():
    if len(sys.argv) != 2 or sys.argv[1] == "-h" or sys.argv[1] == "--help":
        print_help()
        return 1
    filename = sys.argv[1]
    if not os.path.isfile(filename):
        print "ERROR: Invalid File\n"
        print_help()
        return 2
    requests.packages.urllib3.disable_warnings()
    influx_conf, modbus_conf = parse_configuration(filename)
    client = InfluxDBClient(influx_conf[0], influx_conf[1], influx_conf[3], influx_conf[4],
                            influx_conf[2], ssl=True)

    while True:
        for i in range(0, int(influx_conf[5])):
            work_thread = threading.Thread(target=data_worker,
                                           args=(client, i, int(influx_conf[5]), modbus_conf))
            work_thread.start()
        time.sleep(int(influx_conf[6])-time.time()%int(influx_conf[6]))
    return 0

if __name__ == "__main__":
    main()